%md
####Look into external tools 
- Objective - Look into read, write, delete operations on Storage for external tables ONLY.   To identify good candidates that can be migrated to Managed Tables.
  - Identify if an external table is being leveraged by an external platform. If it is it would not be a good candidate until the write operations subside.

External Table -> main.default.mytable
- Leveraged by Databricks
- Leveraged by External Platforms, Read Only for Good Candidate

External Table -> main.default.xyz
- Leveraged by Databricks
- Good Candidate would be read, write, delete

In [0]:
-- Get the latest timestamp for the "eventhub_storage_log_setup" job
with last_run_timestamp as (
  select
    max(period_start_time) as last_run_timestamp
  from
    system.lakeflow.job_run_timeline
  where
    job_id = '1072068050595527'
)

-- Identify list of tables that are good candidates to move to managed tables
select
  src.Storage_AccountName,
  -- max(src.Event_Log_TS) as Event_Log_TS,
  last_run_timestamp.last_run_timestamp as Job_Run_TS,
  src.IT_TableName,

  -- Get read, write, and delete counts
  coalesce(sum(case when src.Storage_Action = 'read' then src.operation_count else 0 end), 0) as total_read_count,
  coalesce(sum(case when src.Storage_Action = 'write' then src.operation_count else 0 end), 0) + coalesce(sum(case when src.Storage_Action = 'delete' then src.operation_count else 0 end), 0) as write_delete_count,

  -- Capture external tables with no writes or deletes as good candidates
  case
    when coalesce(sum(case when src.Storage_Action = 'write' then src.operation_count else 0 end), 0) + coalesce(sum(case when src.Storage_Action = 'delete' then src.operation_count else 0 end), 0) = 0 then 1
    else 0
  end as good_candidate,

  -- TO DO: Capture non-externally accessed databricks tables with any operation
  
  -- Identify external tables that were accessed by an external platform
  case
    when
      sum(
        case
          when lower(src.Storage_userAgentHeader) like '%databricks%' then 1
          else 0
        end
      ) > 0
    then
      0
    else 1
  end as from_external_platform

 -- Group and Filter
from
  (
    select
      Storage_AccountName,
      -- Max(Storage_Time) `Event_Log_TS`,
      Date(Storage_Time) `Storage_Date`,
      lower(Storage_userAgentHeader) as Storage_userAgentHeader,
      IT_TableName,
      IT_TableType,
      Storage_Action,
      count(*) as operation_count
    from
      slog.default.vw_storagelogs_information_schema
    where
      IT_TableType = 'EXTERNAL' and
      Storage_Action in ('read', 'write', 'delete')
    group by
      Storage_AccountName,
      Storage_Time,
      Storage_userAgentHeader,
      IT_TableName,
      IT_TableType,
      Storage_Action
  ) src
  join last_run_timestamp
group by
  src.Storage_AccountName,
  src.IT_TableName,
  last_run_timestamp.last_run_timestamp
order by
  total_read_count desc,
  write_delete_count desc

Storage_AccountName,IT_TableName,IT_TableType,from_external_platform,Storage_OperationName,operation_count
stsezsandbox07,main.default.my_table,EXTERNAL,0,GetPathStatus,15
stsezsandbox07,pos_dev.retailer_na.pos_snapshots,EXTERNAL,0,GetBlob,13
stsezsandbox07,main.default.my_table,EXTERNAL,0,ReadFile,9
stsezsandbox07,main.default.my_table,EXTERNAL,0,FlushFile,6
stsezsandbox07,main.default.my_table,EXTERNAL,0,CreatePathFile,6
stsezsandbox07,main.default.my_table,EXTERNAL,0,AppendFile,6
stsezsandbox07,pos_dev.retailer_na.pos_generator,EXTERNAL,0,GetBlob,4
stsezsandbox07,main.default.my_table,EXTERNAL,0,RenamePathFile,4
stsezsandbox07,pos_dev.retailer_na.pos_static,EXTERNAL,0,GetBlob,3
stsezsandbox07,main.default.my_table,EXTERNAL,0,GetBlob,1


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

%md
#View of Delta Tables not registered in Unity Catalog

Review all delta table storage paths that are not registred in Unity Catalog and see what operations are being done.

In [0]:
-- DELTA_LOG INTERROGATION OF PATHS
with external_tables_not_registered as (
  select
    Storage_AccountName,
    REGEXP_REPLACE(Storage_RelativePath, '^[^/]+://[^/]+/$', '') as Storage_RelativePath,
    Storage_Action,
    count(*) as operation_count
  from
    slog.default.vw_storagelogs_information_schema
  where
    IT_TableType is null
    and lower(REGEXP_REPLACE(Storage_RelativePath, '^[^/]+://[^/]+/$', '')) like '%/_delta_log'
  group by
    Storage_AccountName,
    REGEXP_REPLACE(Storage_RelativePath, '^[^/]+://[^/]+/$', ''),
    Storage_Action
  order by
    operation_count desc
)
select
  *
from
  external_tables_not_registered
pivot (
  max(operation_count) for Storage_Action in ('read' as read_count, 'write' as write_count, 'delete' as delete_count)
)

Storage_AccountName,Storage_RelativePath,IT_TableName,IT_TableType,Storage_OperationName,operation_count
stsezsandbox07,/stsezsandbox07/unity,null,null,ListFilesystemDir,911
stsezsandbox07,/stsezsandbox07/unity,null,null,ListBlobs,555
stsezsandbox07,/stsezsandbox07/pos-dev,null,null,ListBlobs,320
stsezsandbox07,/stsezsandbox07/ps-dev,null,null,ListBlobs,72
stsezsandbox07,/stsezsandbox07/$root,null,null,GetContainerProperties,57
stsezsandbox07,/stsezsandbox07/unity/83ad2d11-1431-4eb5-8a3e-a26542faf5ac/tables/9ba815a2-5ecc-41ee-9f15-7ca779070272/_delta_log,null,null,GetPathStatus,40
stsezsandbox07,/stsezsandbox07/slog,null,null,ListBlobs,31
stsezsandbox07,/stsezsandbox07/unity/83ad2d11-1431-4eb5-8a3e-a26542faf5ac/tables/794d4d99-16ab-4bb9-a9e4-c58c5cde0829/_delta_log,null,null,GetPathStatus,30
stsezsandbox07,/stsezsandbox07/slog,null,null,ListFilesystemDir,27
stsezsandbox07,/stsezsandbox07/retail-demo,null,null,ListBlobs,21
